# EPOBOL and Static Opaque Predicate

## EPOBOL

You may be wondering what is [`EPOBOL`](https://github.com/niosega/EPOBOL). Please, be reassured, there is no link between `EPOBOL` and `COBOL`, eventhough `COBOL` is an interesting language.

`EPOBOL` is the name of my code obfuscator. It stands for `Educative Purpose Obfuscator Based On LLVM`. The aim of a code obfuscator is to make the reverse engineering of a compile code impossible. As soon as nothing is impossible in computer science, a code obfuscator try to modify a program in a way that keep the original logic and that produce a hard-to-understand binary.

There is already a working obfuscator for LLVM, which is called [obfuscator-llvm](https://github.com/obfuscator-llvm/obfuscator/wiki). My goal is not to defeat obfuscator-llvm. My goal is only to understand obfuscation techniques and implement everything by myself to be more and more familiar with the LLVM infrastructure. I strongly recommend the reader to read the wiki of obfuscator-llvm.

I will try to write one article for each obfuscation techniques implemented in `EPOBOL`. For now, there is only one : Static Opaque Predicate (SOP).

## Static Opaque Predicate

The commit that introduce `SOP` to `EPOBOL` is [this one](https://github.com/niosega/EPOBOL/commit/5e33fe563fd822a26bfa3867c91b340715bfbf07). This is a very basic implementation with a lot of limitations. 

### Definition

An Opaque predicate is a predicate that is always true/false during one execution of the program no matter what happened. 

A Static Opaque Predicate(SOP) is a opaque predicate that always have the same value. For example, `x*x > 0` is always evaluated to true no matter the value of `x`.

There are others types of opaque predicate, but we will discuss about it later.

### Principle

To make the reverse engineering process difficult, one solution is to add mess, id est to modify the [Control Flow Graph](https://en.wikipedia.org/wiki/Control_flow_graph)(CFG) to add unecessary branching / dead code.

Suppose that we are trying to obfuscate this code.

In [ ]:
int main() {
    int a = 17;
    printf("%d\n", a);
}

The principle of adding opaque predicate is to add unecessary if/then branching. For instance, an obfuscated version of this code can be :

In [ ]:
int main() {
    int a = 17;
    if (a*a > 0) {
        printf("%d\n", a);
    }
}

### Implementation

LLVM comes with multiples concepts :
* Instruction: Nothing more than one line of LLVM IR.
* Basic Block: A group of instructions such that a block has one entry instruction and one exit instruction.

Example:

In [ ]:
int i = 0;
do{
    i++;
} while(i < 10);
printf("%d", i);

Can be viewed as :

In [ ]:
-----------
Block 1:
int i = 0;
-----------
    |
    |<---------|
    v          |
-----------    | if i < 10
Block 2:       |
i++;           |
i < 10 ? -------
-----------    |
               | if i >= 10
    |-----------
    v
----------
Block 3:
printf("%d", i);
----------

* Pass: A Pass is a transformation or analysis made on the input code. We can have a Basic Block pass, which iterate over Basic Block, Instruction Pass which iterate over Instruction.

To implement `SOP` mecanism, I choose Instruction Pass. Id est, I add a useless if branching at each instruction. To have a good ratio between performance and obfuscation I add a random mecanism that randomly select instruction that will be obfuscated.

But suddently I realize that my approch was wrong. I can not add an if branching where I want. Let take an example in C to illustrate.

In [ ]:
int a = 0;
printf("%d", a);

If every instruction is obfuscated :

In [ ]:
if (...) {
    int a = 0;
}

if (...) {
    printf("%d", a);
}

But hey ! This does not work ! Depending on how the if behave, we may try to print `a` without declaring/initializing it. Mh. That's bad.

I found a very restrictive solution. Never obfuscate instruction that defines a new variable, which in terms of LLVM is translated to :

In [ ]:
if (I->isTerminator() || isa<AllocaInst>(*I) || isa<LoadInst>(*I)
     || isa<CmpInst>(*I) || isa<CallInst>(*I) || isa<GetElementPtrInst>(*I)) {
    continue;
}

This is a very restrictive solution and I have to find something better. I am working on it. To counterbalance this restriction, I obfuscate all others instructions. No more random. (The random mecanism is still in place but useless)

### Predicate choice

We need to find a predicate that is non-trivially true/false, otherwise the compiler will optimize it and get rid of our unecessary if branching. One of the biggest issue when dealing with `SOP` is to find a non trivial predicate. There are a lot of very basic predicate like `true`, `x*x > 0`. [This article](https://faculty.ist.psu.edu/wu/papers/opaque-isc16.pdf) gives a more complex one. Of course, there are an infinite number of such predicate.

Another big issue is to find this `x`. For now, we keep this `x` value as a magic number. But, when inserting the if branching, we must create a `SOP` around a real `x` value. In my implementation, I choose to take the adress of the function where the instruction is in. This adress is not always the same, so the compiler can not optimize it.

In fact, this kind of predicate is very bad, and we'll see in the next section why.

### Predicate detection

The advantage of a `SOP` is that we know that the value is constant. The disadvantage of a `SOP` is that we know that the value is constant. This mean that with a simple fuzzing test during different execution of the progam, we can identify branching that are likely to be a `SOP`. In real life, `SOP` are useless because some tools are now very efficient at detecting and removing this kind of predicate. For example, the [Loop suite](https://faculty.ist.psu.edu/wu/papers/loop-ccs2015.pdf).

### Performance and efficiency

Something really important in obfuscation is to have a good ratio between impact in term of performance and quality of obfuscation. One very simple metric is to count the number of vertices and edges of the CFG. The bigger this number is, the better it is. My next work is to code a benchmark infrastructure that will monitor the compilation, obfuscation and execution of different benchmark and compare the execution between obfuscated and non-obfuscated and try to give a score to the obfuscator.